In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
import os
from dotenv import load_dotenv
load_dotenv()

BUSINESS_ANALYST = "BusinessAnalyst"
SOFTWARE_ENGINEER = "SoftwareEngineer"
PRODUCT_MANAGER = "ProductManager"



kernel = Kernel()



# Create agents for different roles
code_agent = ChatCompletionAgent(
    name=SOFTWARE_ENGINEER, 
    kernel=kernel,
    instructions= """
    You are a professional Software Engineer. Your task is to write efficient, well-structured HTML, CSS, and JS code based on clear requirements.
    All code should be in one code block, and you should not include any explanations or comments.
    Output clean code only, enclosed within appropriate tags. Do not include explanations unless explicitly asked.   
    """
    ) 
business_analyst_agent = ChatCompletionAgent(
    name=BUSINESS_ANALYST, 
    kernel=kernel,
    instructions=""""
    You are a skilled Business Analyst. Based on the given instructions, your task is to view the requirements and create 
    a clear and concise summary of the requirements.
    Your summary should be structured, highlighting key points and ensuring clarity for next agent.
    Do not include any code or technical details, just a summary of the requirements.
    """
    
    )
product_managet_agent = ChatCompletionAgent(
    name=PRODUCT_MANAGER, 
    kernel=kernel,
    instructions=""""
     You are the Product Owner. Your job is to ensure that the Software Engineer's code meets all of the user's original requirements.

Your responsibilities:
- Cross-check the HTML implementation against the Business Analyst’s requirements.
- Ensure all features are present and functioning as described. If yes please acknowledge with "APPROVED"
- If any requirements are missing or not implemented correctly, provide specific feedback on what needs to be fixed.
- Confirm that the Software Engineer has shared the code using this required format:
   ```html
   <!-- HTML content -->
    """
    
    )




selection_function = KernelFunctionFromPrompt(
    function_name="selection", 
    prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {BUSINESS_ANALYST}
- {SOFTWARE_ENGINEER}
- {PRODUCT_MANAGER}

Rules:
- If RESPONSE is user input, it is {BUSINESS_ANALYST}'s turn.
- If RESPONSE is by {BUSINESS_ANALYST}, it is {SOFTWARE_ENGINEER}'s turn.
- If RESPONSE is by {SOFTWARE_ENGINEER}, it is {PRODUCT_MANAGER}'s turn.
- If RESPONSE is by {PRODUCT_MANAGER} and it does not meet all the requirements, it is {SOFTWARE_ENGINEER}'s turn.
- If RESPONSE is by {PRODUCT_MANAGER} and it meets all the requirements, Return RESPONSE with APPROVE Keyword.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

termination_keyword = "APPROVE"

termination_function = KernelFunctionFromPrompt(
    function_name="termination", 
    prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
If specific suggestions are being provided, it is not satisfactory.
If no correction is suggested, it is satisfactory.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

history_reducer = ChatHistoryTruncationReducer(target_count=1)

chat = AgentGroupChat(
    agents=[business_analyst_agent, code_agent, product_managet_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=business_analyst_agent,
        function=selection_function,
        kernel=kernel,
        #result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[product_managet_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)
is_complete: bool = False
while not is_complete:
    print()
    user_input = input("User > ").strip()
    if not user_input:
        continue
    
    if user_input.lower() == "exit":
        is_complete = True
        break
    
    if user_input.lower() == "reset":
        await chat.reset()
        print("[Conversation has been reset]")
        continue
    
    # Try to grab files from the script's current directory
    if len(user_input) > 1:
        await chat.add_chat_message(message=user_input)

In [ ]:
REVIEWER_NAME = "Reviewer"
WRITER_NAME = "Writer"


agent_reviewer = ChatCompletionAgent(
        kernel=kernel,
        name=REVIEWER_NAME,
        instructions="""
Your responsibility is to review and identify how to improve user provided content.
If the user has provided input or direction for content already provided, specify how to address this input.
Never directly perform the correction or provide an example.
Once the content has been updated in a subsequent response, review it again until it is satisfactory.

RULES:
- Only identify suggestions that are specific and actionable.
- Verify previous suggestions have been addressed.
- Never repeat previous suggestions.
""",
)

agent_writer = ChatCompletionAgent(
        kernel=kernel,
        name=WRITER_NAME,
        instructions="""
Your sole responsibility is to rewrite content according to review suggestions.
- Always apply all review directions.
- Always revise the content in its entirety without explanation.
- Never address the user.
""",
    )


selection_function = KernelFunctionFromPrompt(
    function_name="selection", 
    prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {REVIEWER_NAME}
- {WRITER_NAME}

Rules:
- If RESPONSE is user input, it is {REVIEWER_NAME}'s turn.
- If RESPONSE is by {REVIEWER_NAME}, it is {WRITER_NAME}'s turn.
- If RESPONSE is by {WRITER_NAME}, it is {REVIEWER_NAME}'s turn.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
    function_name="termination", 
    prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
If specific suggestions are being provided, it is not satisfactory.
If no correction is suggested, it is satisfactory.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

history_reducer = ChatHistoryTruncationReducer(target_count=1)

chat = AgentGroupChat(
    agents=[agent_reviewer, agent_writer],
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=agent_reviewer,
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[agent_reviewer],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)




In [ ]:
is_complete: bool = False
while not is_complete:
    print()
    user_input = input("User > ").strip()
    if not user_input:
        continue
    
    if user_input.lower() == "exit":
        is_complete = True
        break
    
    if user_input.lower() == "reset":
        await chat.reset()
        print("[Conversation has been reset]")
        continue
    
    # Try to grab files from the script's current directory
    if user_input.startswith("@") and len(user_input) > 1:
        file_name = user_input[1:]
        print(file_name)
        #script_dir = os.path.dirname(os.path.abspath(__file__))
        file_path = "./azsemantickernal/WomensSuffrage.txt"
        try:
            #if not os.path.exists(file_path):
            #    print(f"Unable to access file: {file_path}")
            #    continue
            with open(file_path, "r", encoding="utf-8") as file:
                user_input = file.read()
        except Exception:
            print(f"Unable to access file: {file_path}")
            continue

# Add the current user_input to the chat
await chat.add_chat_message(message=user_input)

In [ ]:
try:
    async for response in chat.invoke():
        if response is None or not response.name:
            continue
        print()
        print(f"# {response.name.upper()}:\n{response.content}")
except Exception as e:
    print(f"Error during chat invocation: {e}")

# Reset the chat's complete flag for the new conversation round.
chat.is_complete = False

In [ ]:

import asyncio
import os

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.functions import KernelFunctionFromPrompt
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define agent names
REVIEWER_NAME = "Reviewer"
WRITER_NAME = "Writer"



chat_completion_service = AzureChatCompletion(service_id="my-service-id")

kernel = Kernel()
kernel.add_service(service=chat_completion_service)

from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = OpenAIChatPromptExecutionSettings()

chat_history = ChatHistory()
chat_history.add_user_message("Hello, how are you?")

response = await chat_completion_service.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

async for chunk in response:
    print(chunk, end="")